# SE (Squeeze-and-Excitation) Attention (CV) — Ders Notu

Bu defter, **SE (Squeeze-and-Excitation)** mekanizmasını görüntü işleme odaklı olarak açıklar:  
- Ne problemi çözer?  
- Matematiksel olarak nasıl çalışır?  
- CNN bloklarına nerede ve nasıl eklenir?  
- Pratikte hangi durumlarda faydalıdır / değildir?

> Not: Kod kısmı temel düzeyde tutulmuştur. Uygulama/optimizasyon detaylarına daha sonra birlikte girilebilir.


## 1) Neden SE? (Motivasyon)

CNN'lerde bir katman çıktı feature map'i genelde şu şekildedir:

- **X ∈ ℝ^(B×C×H×W)**  
  - **B**: batch  
  - **C**: kanal sayısı  
  - **H×W**: uzamsal boyut

Standart Conv blokları uzamsal desenleri öğrenir; ancak her kanalın **ne kadar önemli** olduğu her örnek için değişebilir.

SE'nin hedefi:

- **“Bu girişte hangi kanallar daha önemli?”** sorusuna cevap vererek kanalları **yeniden ağırlıklandırmak**.

Bu sayede model, gereksiz/az faydalı kanalları baskılar; faydalı kanalları güçlendirir.


## 2) SE'nin Temel Fikri (Özet Akış)

SE üç adımda çalışır:

1. **Squeeze (Sıkıştırma):**  
   Uzamsal bilgiyi toplayıp her kanal için tek bir özet değer çıkarır (Global Average Pooling).

2. **Excitation (Uyarma):**  
   Bu kanal özetlerinden bir **kapılayıcı (gating)** fonksiyon üretir: her kanal için **0–1 arası ağırlık**.

3. **Scale (Ölçekleme):**  
   Feature map'i kanal bazında çarparak yeniden ölçekler.


## 3) Matematiksel Tanım

Girdi: **X ∈ ℝ^(B×C×H×W)**

### 3.1 Squeeze
Her kanal için global ortalama:

\[
s_c = \frac{1}{H\cdot W}\sum_{i=1}^{H}\sum_{j=1}^{W} X_{c,i,j}
\]

Bu işlem sonucu:

- **s ∈ ℝ^(B×C)** (veya pratikte **B×C×1×1**)

### 3.2 Excitation
Kanal özetinden kanal ağırlığı üretmek için küçük bir MLP kullanılır:

\[
z = \sigma\left(W_2 \cdot \delta(W_1 \cdot s)\right)
\]

- \(W_1: \mathbb{R}^{C} \to \mathbb{R}^{C/r}\)  
- \(W_2: \mathbb{R}^{C/r} \to \mathbb{R}^{C}\)  
- \(r\): reduction ratio (genelde 4, 8, 16)  
- \(\delta\): ReLU gibi aktivasyon  
- \(\sigma\): Sigmoid (çıktı 0–1)

Sonuç:

- **z ∈ ℝ^(B×C)** (kanal ağırlıkları)

### 3.3 Scale
Kanal ağırlıklarını feature map'e uygula:

\[
Y_{c,i,j} = z_c \cdot X_{c,i,j}
\]

Burada \(z_c\) her kanal için tek bir skalar olup H×W boyunca broadcast edilir.


## 4) SE Blok Nerede Kullanılır?

SE genelde şu bölgelerde kullanılır:

- **Residual blok içinde** (ResNet/ResNeXt türevleri)  
- **Bottleneck bloklarda**  
- **Mobil bloklarda** (MobileNetV3 gibi)  
- CNN backbone'larında **Conv → BN → Activation** sonrasında

Pratik şablonlar:
- `Conv-BN-ReLU -> SE -> Conv-BN -> Residual Add -> ReLU`
- veya `Conv-BN-ReLU -> Conv-BN-ReLU -> SE`

Genel kural: SE, bir bloğun ürettiği kanalları “kalibre etmek” için bloğun sonlarına yakın koyulur.


## 5) Parametre ve Maliyet (Kaba Hesap)

SE'nin parametre yükü, küçük MLP'den gelir.

- \(W_1\): \(C \times (C/r)\)  
- \(W_2\): \((C/r) \times C\)

Toplam (bias hariç):

\[
2\cdot \frac{C^2}{r}
\]

Örnek: \(C=128, r=16\) için:

\[
2\cdot \frac{128^2}{16} = 2048
\]

Bu, tipik bir 3×3 conv parametresine göre oldukça küçük kalır.

Not: Hesaplama maliyeti de düşüktür; çünkü SE uzamsal boyutta çalışmaz (H×W üzerinde ağır işlem yok).


## 6) SE'nin Güçlü ve Zayıf Yanları

### Güçlü Yanlar
- Kanal önemini örneğe göre adaptif ayarlar
- Parametre ve FLOP ek yükü genellikle düşük
- Birçok backbone'da “drop-in” gibi eklenebilir
- Çoğu zaman doğruluk/kararlılık iyileştirir

### Zayıf Yanlar / Dikkat
- SE **spatial attention değildir**: “neresi önemli?”yi değil, “hangi kanal önemli?”yi öncelikler
- Çok küçük C değerlerinde (örn. C=8,16) reduction agresifse bilgi kaybı olabilir
- Bazı mobil senaryolarda latency (özellikle CPU) küçük de olsa artabilir


## 7) SE, ECA, CBAM Arasındaki Fark (Kısa)

- **SE:** Kanal ağırlığını MLP ile öğrenir (global avg pool + 2 FC).  
- **ECA:** MLP yerine kanal boyutunda hafif 1D conv ile daha ucuz kanal attention.  
- **CBAM:** Kanal attention + spatial attention birlikte.

Bu yüzden sıralamada genelde: **SE/ECA → CBAM** ilerlemek mantıklıdır.


## 8) Minimal PyTorch Şablonu (Temel)

Aşağıdaki kod, SE’nin temel yapısını gösterir.  
Amaç: Mekanizmayı görmek; performans/optimizasyon daha sonra ele alınabilir.

In [4]:
import torch
import torch.nn as nn

class SEBlock(nn.Module):
    def __init__(self, channels: int, reduction: int = 16):
        super().__init__()
        hidden = max(1, channels // reduction)
        self.pool = nn.AdaptiveAvgPool2d(1)  # (B,C,H,W) -> (B,C,1,1)
        self.fc = nn.Sequential(
            nn.Conv2d(channels, hidden, kernel_size=1, bias=True),
            nn.ReLU(inplace=True),
            nn.Conv2d(hidden, channels, kernel_size=1, bias=True),
            nn.Sigmoid()
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        s = self.pool(x)
        w = self.fc(s)          # (B,C,1,1)
        return x * w            # channel-wise scale


## 9) SE'yi Bir Conv Bloğa Takma (Temel Örnek)

In [5]:
class ConvBNAct(nn.Module):
    def __init__(self, cin, cout, k=3, s=1, p=1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(cin, cout, k, stride=s, padding=p, bias=False),
            nn.BatchNorm2d(cout),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.net(x)

class ConvSEBlock(nn.Module):
    def __init__(self, cin, cout, reduction=16):
        super().__init__()
        self.conv = ConvBNAct(cin, cout, k=3, s=1, p=1)
        self.se = SEBlock(cout, reduction=reduction)

    def forward(self, x):
        y = self.conv(x)
        y = self.se(y)
        return y


## 10) Hızlı Doğrulama (Şekil Kontrolü)

In [6]:
x = torch.randn(2, 64, 56, 56)
m = ConvSEBlock(64, 128, reduction=16)
y = m(x)
x.shape, y.shape


(torch.Size([2, 64, 56, 56]), torch.Size([2, 128, 56, 56]))

## 11) Hangi Modellerde Nerede Kullanılır? (Pratik)

### Classification
- Backbone bloklarına (ResNet/ConvNeXt benzeri) eklenebilir.
- Özellikle orta/derin katmanlarda kanal kalibrasyonu fayda sağlar.

### Detection / Segmentation
- Backbone ve neck tarafında faydalı olabilir.
- Ancak “konum/uzamsal seçicilik” gerekiyorsa SE tek başına sınırlıdır; CBAM/CoordAtt gibi ekler düşünülebilir.

### Mobil / Edge
- SE bazen uygun, bazen ECA daha iyi bir “maliyet/latency” dengesi verir.


## 12) Tipik Hiperparametreler ve Seçim

- **reduction (r):** 4 / 8 / 16 en yaygın.  
  - C büyükse r=16 genelde güvenli  
  - C küçükse r'yi küçült (örn. r=4)  
- **Aktivasyon:** ReLU (standart), bazı tasarımlarda SiLU/Swish de görülür  
- **Pool:** Genelde AvgPool; MaxPool kullanan varyantlar da var ama SE'nin klasik hali AvgPool'dur.


## 13) Sık Yapılan Hatalar

- SE ağırlıklarını sigmoid yerine softmax yapmak: farklı davranış verir; klasik SE sigmoid'dir.
- reduction çok büyük seçmek (C küçükken): hidden=0/1'e sıkışır; öğrenme zayıflar.
- SE’yi yanlış yere koymak: genelde blok çıktısına yakın koymak daha verimli olur.


## 14) Deney Tasarımı (Ablation Önerisi)

SE'yi değerlendirirken şu ablation'lar temiz sonuç verir:

1) Baseline backbone (SE yok)  
2) Backbone + SE (r=16)  
3) Backbone + SE (r=8)  
4) Backbone + ECA (karşılaştırma)  
5) Backbone + CBAM (kanal+spatial)

Metrikler:
- Accuracy/mAP/IoU (göreve göre)
- Params, MACs, latency (profiling ile)


## 15) Kısa Özet

- SE, **kanal bazlı attention** mekanizmasıdır.  
- Uzamsal bilgi global ortalama ile sıkıştırılır, küçük bir MLP ile kanal ağırlıkları üretilir.  
- Genelde düşük maliyetle performansı artırabilir.  
- Spatial seçicilik gerekiyorsa CBAM/CoordAtt gibi yöntemlerle tamamlanabilir.


## Kaynak Notu

Araştırma için anahtar kelimeler:
- “Squeeze-and-Excitation Networks”
- “Channel attention CNN”
- “SE block ResNet integration”
